$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{Universidad de los Andes}$$
$$\textrm{Grupo de Física de Altas Energías: Fenomenología de Partículas}$$

$\textbf{Preliminares}$ 

Las librerías que se usan en este capítulo son las siguientes: 

In [1]:
import os, sys

Path_Tutorials = os.path.dirname(os.path.realpath('Capitulo_6_Pureza_y_significancia.ipynb'))
Path_Pheno_BSM = os.path.dirname(Path_Tutorials)
sys.path.append(Path_Pheno_BSM)

from delphes_reader import DelphesLoader

import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.10g}'.format) #Los DataFrame se imprimen hasta con 10 decimales (así no se pone notación científica)

from IPython.display import clear_output #Permite limpiar la línea del output para que no se imprima nada (es algo estético que se usa a veces) 
import nbimporter #Permite importar otros notebooks y usar sus funciones

personal_folder = f'{Path_Pheno_BSM}/Tutorials'

Welcome to JupyROOT 6.26/06


En este capítulo se utilizan la lista signals, el directorio XS y la función Eficiencia, por esto es necesario volverlos a definir para no tener inconvenientes:

In [2]:
#Definamos una lista con las señales y un directorio para guardar las secciones eficaces:

signals = ["z", "w_jets", "ww", "wz", "zz", "ttbar", "stop"]
XS = {}

#XS de señal:
tabla_ij = pd.read_html(f'{personal_folder}/Data_Z/crossx.html')
columna_xs = tabla_ij[0]['Cross section (pb)']
XS['z'] = float(columna_xs[0].split(' ')[0])    

#XS de backgrounds:
BKGs = ["w_jets", "ww", "wz", "zz", "ttbar", "stop"]

for BKG in BKGs:
    Delphes_Process = DelphesLoader(BKG) #Se crea un objeto de la clase DelphesLoader correspondiente al proceso BKG
    XS[BKG] = float(Delphes_Process.xs) #Extrae la sección eficaz del BKG -  esta en pb
clear_output(wait=False)

In [3]:
#Importemos las funciones Eficiencia y Luminosidad del capitulo 3
from Capitulo_3_Guardado_de_los_histogramas_en_un_archivo_root import Eficiencia 
from Capitulo_3_Guardado_de_los_histogramas_en_un_archivo_root import Luminosidad

Ahora lo que se debe hacer es analizar el archivo cut_flows.csv y calcular la significancia y la pureza de los datos. En este tutorial usaremos las siguientes expresiones para esto:

$$ \textbf{Pureza} = \frac{\textrm{Número de Datos de señal}}{\textrm{Número de Datos de señal + Número de Datos de Background}} $$

$$ \textbf{Significancia} = \frac{\textrm{Número de Datos de señal}}{ \sqrt{\textrm{Número de Datos de señal + Número de Datos de Background}}} $$

Para esto carguemos y analicemos cut_flows:

In [4]:
cut_flows = pd.read_csv(f'{personal_folder}/CSV_Z_Analisis/cut_flows_actualizado.csv',index_col = 0)
pd.DataFrame(cut_flows)

,Total,z,w_jets,ww,wz,zz,ttbar,stop
Todos,89400073,300000,20942823,12500000,9850000,10000000,24307250,11500000
Al menos 2 muones,936808,210853,104,74404,160145,299277,170215,21810
Exactamente 2 muones,916832,210828,97,74363,148789,291091,169923,21741
Carga opuesta,907174,210821,77,74288,142757,289570,168424,21237
p_T[0] > 30 GeV,853390,190460,26,66691,138989,282760,154900,19564
p_T[1] > 20 GeV,753660,183580,10,54154,123123,251079,125848,15866
|Eta| < 2.4,753660,183580,10,54154,123123,251079,125848,15866
DeltaR > 0.3,751645,183580,6,53657,123001,250981,124738,15682
66 (GeV) < M(Z) < 116 (Gev),620460,180556,2,20741,119827,249272,44512,5550


Este DataFarme contiene el número de eventos Montecarlo, ahora construyamos un DataFrame similar donde cada entrada corresponda el número de eventos físicos (es decir, el número de eventos que usabamos para normalizar los histogramas anteriormente).

In [5]:
cut_flows = pd.DataFrame(cut_flows)
Total_filas, Total_columnas = np.shape(cut_flows)

N_events = pd.DataFrame()

for indice_fila in range(Total_filas):
    N_events_matrix = {}
    for i in range(1,len(cut_flows.keys())):
        
        signal = cut_flows.keys()[i]
        #Revisemos que esa columna corresponda a las señales: "z", "w_jets", etc, así se ignora la columna "Total" esa no tiene sentido analizarla
        if signal in signals:
            N = Eficiencia(signal, Path_CSV = personal_folder, cut_flows_actualizado=True, fila = indice_fila)*Luminosidad()*XS[signal]
            N_events_matrix[signal] = {'N': N} #guardarlos con la etiqueta "N" es un testigo para la construcción del DataFrame
            
    N_events = pd.concat([N_events, pd.DataFrame(N_events_matrix)])
N_events.index = cut_flows.index #Pone el mismo index de cut_flows, es decir, la etiqueta que permite identificar la fila a que corte corresponde

In [6]:
N_events

,z,w_jets,ww,wz,zz,ttbar,stop
Todos,6372000,1465483000,655073.6,246918.4,95624.16,5046960,2411700
Al menos 2 muones,4478517.72,7277.444497,3899.207691,4014.492098,2861.811173,35342.06035,4573.841478
Exactamente 2 muones,4477986.72,6787.616502,3897.059049,3729.821504,2783.533236,35281.43184,4559.371278
Carga opuesta,4477838.04,5388.107945,3893.128608,3578.612186,2768.988801,34970.19165,4453.675904
p_T[0] > 30 GeV,4045370.4,1819.361124,3495.001077,3484.156497,2703.868748,32162.17812,4102.825983
p_T[1] > 20 GeV,3899239.2,699.7542786,2837.988459,3086.429864,2400.921847,26130.05676,3327.307148
|Eta| < 2.4,3899239.2,699.7542786,2837.988459,3086.429864,2400.921847,26130.05676,3327.307148
DeltaR > 0.3,3899239.2,419.8525672,2811.942732,3083.371586,2399.98473,25899.58537,3288.719948
66 (GeV) < M(Z) < 116 (Gev),3835009.44,139.9508557,1086.950523,3003.806205,2383.642561,9242.110215,1163.907391


Finalmente, con N_events se puede calcular la pureza y la significancia para cada corte, hagámoslo y añadamos una columna para cada cantidad.

In [7]:
def pureza_significancia(Fila, N_events):
    
    matriz = np.asmatrix(N_events)
    Num_Data_signal = matriz[Fila, 0]
    Num_Data_BKG = np.sum(matriz[Fila, 1:])
    
    pureza = Num_Data_signal/(Num_Data_signal + Num_Data_BKG)
    significancia = Num_Data_signal/np.sqrt(Num_Data_signal + Num_Data_BKG)
    
    return pureza, significancia

In [8]:
purezas = []
significancias = []

for indice_fila in range(Total_filas):

    pureza, significancia = pureza_significancia(indice_fila, N_events)
    purezas.append(pureza)
    significancias.append(significancia)

N_events["Pureza"] = purezas
N_events["Significancia"] = significancias

Así,

In [9]:
pd.DataFrame(N_events)

,z,w_jets,ww,wz,zz,ttbar,stop,Pureza,Significancia
Todos,6372000,1465483000,655073.6,246918.4,95624.16,5046960,2411700,0.004304500076,165.6148377
Al menos 2 muones,4478517.72,7277.444497,3899.207691,4014.492098,2861.811173,35342.06035,4573.841478,0.9872216403,2102.686284
Exactamente 2 muones,4477986.72,6787.616502,3897.059049,3729.821504,2783.533236,35281.43184,4559.371278,0.9874225993,2102.775615
Carga opuesta,4477838.04,5388.107945,3893.128608,3578.612186,2768.988801,34970.19165,4453.675904,0.9878548352,2103.200884
p_T[0] > 30 GeV,4045370.4,1819.361124,3495.001077,3484.156497,2703.868748,32162.17812,4102.825983,0.9883298843,1999.540062
p_T[1] > 20 GeV,3899239.2,699.7542786,2837.988459,3086.429864,2400.921847,26130.05676,3327.307148,0.9902272274,1964.976545
|Eta| < 2.4,3899239.2,699.7542786,2837.988459,3086.429864,2400.921847,26130.05676,3327.307148,0.9902272274,1964.976545
DeltaR > 0.3,3899239.2,419.8525672,2811.942732,3083.371586,2399.98473,25899.58537,3288.719948,0.9903728515,1965.121026
66 (GeV) < M(Z) < 116 (Gev),3835009.44,139.9508557,1086.950523,3003.806205,2383.642561,9242.110215,1163.907391,0.9955814548,1953.986765
